In [ ]:
import numpy as np
from pathlib import Path
import torch
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
data_dir = Path("data/processed")
x = np.load(data_dir / "x.npy")
y = np.load(data_dir / "y.npy")
y = y[y.sum((1, 2)) != 0]
dts = np.load(data_dir / "dts.npy")
sdts = np.load(data_dir / "sdts.npy")

In [ ]:
import matplotlib.pyplot as plt
import random
import data as dt

dataset = dt.ToyExperiment(x, y, dts, sdts)
x0, y0, dts0, sdts0 = dataset[0:3]
plt.imshow(np.transpose(x0[:3], (1, 2, 0)) / x0.max())
plt.show()
plt.imshow(y0[0])

In [ ]:
import torch.optim
import networks
from train import train
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = networks.backend_cnn("data/models/MS_DeepLab_resnet_trained_VOC.pth")
lr, wd, momentum = 1e-4, 1e-6, 0.9
loaders = {"train": DataLoader(dataset, batch_size=8), "val": DataLoader(dataset, batch_size=16)}
train_params = [{'params': net.get_1x_lr_params(), 'lr': lr},
                {'params': net.get_10x_lr_params(), 'lr': lr * 10}]
optimizer = torch.optim.SGD(train_params, lr=lr, momentum=momentum, weight_decay=wd)
net, optimizer = train(loaders, net, optimizer)